# qiime2-MICOM install + community tutorial 
From Laurie: created a venv using the .yml files for the qiime distribution and q2-micom addition
  - find appropriate qiime distribution .yml [here](https://docs.qiime2.org/2024.5/install/native/) -- I used qiime2-amplicon-2024.5 for macOS (Apple Silicon)
  - [q2-micom.yml](https://raw.githubusercontent.com/micom-dev/q2-micom/main/q2-micom.yml)

In [ ]:
#created a new qiime environment with a different name than my base qiime env
#just wanted to make sure this doesn't mess up my current install
micromamba env -n qiime2-amplicon-micom -f qiime2-amplicon-2024.5-py39-osx-conda.yml  

#activate this environment
micromamba activate qiime2-amplicon-micom 

#micromamba doesn't have an update command to update from .yml file
#use micromamba install instead
#if using conda or mamba, should be able to use update
#e.g. conda update -n qiime2-amplicon-micom -f q2-micom.yml
micromamba install -f q2-micom.yml